# Prepare Workspace

## Install required modules

In [1]:
#!pip install -U -q PyDrive
#!pip install geopandas

## Load required modules

In [1]:
import pandas as pd
import numpy as np

import datetime

import matplotlib.pyplot as plt
import seaborn as sns
#from scipy import stats

#from google.colab import data_table
#data_table.enable_dataframe_formatter()
from sklearn.preprocessing import MinMaxScaler
#from sklearn.linear_model import LinearRegression
#from scipy.optimize import linprog
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error


#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#from google.colab import files

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
def VIF(X, threshold = 5):
  from statsmodels.stats.outliers_influence import variance_inflation_factor
  columns = X.columns.to_list()
  vif = pd.DataFrame()
  vif["VIF Factor"] = [variance_inflation_factor(X.values, i)\
                       for i in range(len(columns))]
  vif["features"] = X.columns

  ## Check for infinity VIF
  mask = (vif["VIF Factor"] == np.infty)
  if mask.any():
    to_drop=vif[mask]["features"].to_list()
    setColumns = set(columns)
    setTo_drop = set(to_drop)
    columns_new = list(setColumns.difference(setTo_drop))
    print(f"dropping due to infinite VIF:")
    print("\n".join(setTo_drop))
    return VIF(X[columns_new], threshold=threshold)

  # Drop feature with highest VIF if exceeding threshold
  if vif["VIF Factor"].max() > threshold:
    index = vif["VIF Factor"].idxmax()
    to_drop = vif.loc[index].to_dict()
    print("dropping {features} at index {index} with VIF of {VIF Factor:.2f} exceeding threshold of {threshold}"\
          .format(**to_drop, index=index, threshold=threshold))
    columns.remove(to_drop["features"])
    return VIF(X[columns], threshold)
  
  return vif

In [3]:
targets       = ['MAX_Completeness_pct', 'Weeks2MAX', 'IDX_W2M']
spatial       = ['Population Density (Per Sq. Mile)', 'Area (Land)', 'Average Commute to Work (In Min)', 'AREA_SQMI', 'Density_Medical_Facilities_Population', 'Density_Medical_Facilities_Area', 'Density_Transportation_Stops_Pop', 'Density_Transportation_Stops_Area', 'Stops_per_Facility']
socioeconomic = ["Total Population","Total Population:","% Total Population: Male","% Total Population: Female","Total Population:.1","% Total Population: Under 5 Years","% Total Population: 5 to 9 Years","% Total Population: 10 to 14 Years","% Total Population: 15 to 17 Years","% Total Population: 18 to 24 Years","% Total Population: 25 to 34 Years","% Total Population: 35 to 44 Years","% Total Population: 45 to 54 Years","% Total Population: 55 to 64 Years","% Total Population: 65 to 74 Years","% Total Population: 75 to 84 Years","% Total Population: 85 Years and Over","% Total Population: White Alone","% Total Population: Black or African American Alone","% Total Population: American Indian and Alaska Native Alone","% Total Population: Asian Alone","% Total Population: Native Hawaiian and Other Pacific Islander Alone","% Total Population: Some Other Race Alone","% Total Population: Two or More Races","% Population 25 Years and Over: Less than High School","% Population 25 Years and Over: High School Graduate or More (Includes Equivalency)","% Population 25 Years and Over: Some College or More","% Population 25 Years and Over: Bachelor's Degree or More","% Population 25 Years and Over: Master's Degree or More","% Population 25 Years and Over: Professional School Degree or More","% Population 25 Years and Over: Doctorate Degree","% Population 16 Years and Over: in Labor Force","% Population 16 Years and Over: in Labor Force: in Armed Forces","% Population 16 Years and Over: in Labor Force: Civilian","% Population 16 Years and Over: in Labor Force: Civilian: Employed","% Population 16 Years and Over: in Labor Force: Civilian: Unemployed","% Population 16 Years and Over: Not in Labor Force","% Households: Less than $25,000","% Households: $25,000 to $49,999","% Households: $50,000 to $74,999","% Households: $75,000 to $99,999","% Households: $100,000 or More","Median Household Income (In 2021 Inflation Adjusted Dollars)","Gini Index","% Workers 16 Years and Over: Car, Truck, or Van","% Workers 16 Years and Over: Drove Alone","% Workers 16 Years and Over: Carpooled","% Workers 16 Years and Over: Public Transportation (Includes Taxicab)","% Workers 16 Years and Over: Motorcycle","% Workers 16 Years and Over: Bicycle","% Workers 16 Years and Over: Walked","% Workers 16 Years and Over: Other Means","% Workers 16 Years and Over: Worked At Home","% Total: No Health Insurance Coverage","% Total: with Health Insurance Coverage","% Total: with Health Insurance Coverage: Public Health Coverage","% Total: with Health Insurance Coverage: Private Health Insurance","E_TOTPOP","EP_POV150","EP_UNEMP","EP_HBURD","EP_NOHSDP","EP_UNINSUR","EP_AGE65","EP_AGE17","EP_DISABL","EP_SNGPNT","EP_LIMENG","EP_MINRTY","EP_MUNIT","EP_MOBILE","EP_CROWD","EP_NOVEH","EP_GROUPQ","EPL_POV150","EPL_UNEMP","EPL_HBURD","EPL_NOHSDP","EPL_UNINSUR","SPL_THEME1","RPL_THEME1","EPL_AGE65","EPL_AGE17","EPL_DISABL","EPL_SNGPNT","EPL_LIMENG","SPL_THEME2","RPL_THEME2","EPL_MINRTY","SPL_THEME3","RPL_THEME3","EPL_MUNIT","EPL_MOBILE","EPL_CROWD","EPL_NOVEH","EPL_GROUPQ","SPL_THEME4","RPL_THEME4","SPL_THEMES","RPL_THEMES","F_POV150","F_UNEMP","F_HBURD","F_NOHSDP","F_UNINSUR","F_THEME1","F_AGE65","F_AGE17","F_DISABL","F_SNGPNT","F_LIMENG","F_THEME2","F_MINRTY","F_THEME3","F_MUNIT","F_MOBILE","F_CROWD","F_NOVEH","F_GROUPQ","F_THEME4","F_TOTAL","E_DAYPOP","EP_NOINT","EP_AFAM","EP_HISP","EP_ASIAN","EP_AIAN","EP_NHPI","EP_TWOMORE","EP_OTHERRACE","%Total Child Population",'%Total University','%Total High school','%Total Less than High School','%Total Adult Population','%Total Elderly Population',"%Population with Disability","%LIMENG","%NOINT"]
others        = ["Qualifying Name","File identification","State Postal Abbreviation","Summary Level","Geographic Component","Logical Record Number","US","Census Region","Census Division","State (Census Code)","State (FIPS Code)","County of current residence","County Subdivision (FIPS)","Place (FIPS Code)","Census Tract","Block Group","Consolidated City","American Indian Area/AlaskaNative Area/ Hawaiian Home Land(Census)","American Indian Area/AlaskaNative Area/ Hawaiian Home Land(FIPS)","American Indian Trust Land/Hawaiian Home Land Indicator","American Indian TribalSubdivision (Census)","American Indian TribalSubdivision (FIPS)","Alaska Native RegionalCorporation (FIPS)","Metropolitan and MicropolitanStatistical Area","Combined Statistical Area","Metropolitan Statistical Area-Metropolitan Division","Metropolitan Area Central City","Metropolitan/MicropolitanIndicator Flag","New England City and Town Area","New England City and TownCombined Statistical Area","New England City and Town AreaDivision","Urban Area","Urban Area Central Place","Current Congressional District ***","State Legislative District Upper","State Legislative District Lower","Voting District","ZIP Code Tabulation Area (3-digit)","5-digit ZIP Code Tabulation Area","Subminor Civil Division (FIPS)","State-School District (Elementary)","State-School District (Secondary)","State-School District (Unified)","Urban/Rural","Principal City Indicator","Traffic Analysis Zone","Urban Growth Area","Public Use Microdata Area â€“ 5%File","Public Use Microdata Area - 1% File","Geographic Identifier","Area Name","Tribal Tract","Tribal Block Group","Place (State FIPS + Place FIPS)","Total Population: Male","Total Population: Female","Total Population: Under 5 Years","Total Population: 5 to 9 Years","Total Population: 10 to 14 Years","Total Population: 15 to 17 Years","Total Population: 18 to 24 Years","Total Population: 25 to 34 Years","Total Population: 35 to 44 Years","Total Population: 45 to 54 Years","Total Population: 55 to 64 Years","Total Population: 65 to 74 Years","Total Population: 75 to 84 Years","Total Population: 85 Years and Over","Total Population:.2","Total Population: White Alone","Total Population: Black or African American Alone","Total Population: American Indian and Alaska Native Alone","Total Population: Asian Alone","Total Population: Native Hawaiian and Other Pacific Islander Alone","Total Population: Some Other Race Alone","Total Population: Two or More Races","Population 25 Years and Over:","Population 25 Years and Over: Less than High School","Population 25 Years and Over: High School Graduate or More (Includes Equivalency)","Population 25 Years and Over: Some College or More","Population 25 Years and Over: Bachelor's Degree or More","Population 25 Years and Over: Master's Degree or More","Population 25 Years and Over: Professional School Degree or More","Population 25 Years and Over: Doctorate Degree","Population 16 Years and Over:","Population 16 Years and Over: in Labor Force","Population 16 Years and Over: in Labor Force: in Armed Forces","Population 16 Years and Over: in Labor Force: Civilian","Population 16 Years and Over: in Labor Force: Civilian: Employed","Population 16 Years and Over: in Labor Force: Civilian: Unemployed","Population 16 Years and Over: Not in Labor Force","Households:","Households: Less than $25,000","Households: $25,000 to $49,999","Households: $50,000 to $74,999","Households: $75,000 to $99,999","Households: $100,000 or More","Workers 16 Years and Over:","Workers 16 Years and Over: Car, Truck, or Van","Workers 16 Years and Over: Drove Alone","Workers 16 Years and Over: Carpooled","Workers 16 Years and Over: Public Transportation (Includes Taxicab)","Workers 16 Years and Over: Motorcycle","Workers 16 Years and Over: Bicycle","Workers 16 Years and Over: Walked","Workers 16 Years and Over: Other Means","Workers 16 Years and Over: Worked At Home","Total:","Total: No Health Insurance Coverage","Total: with Health Insurance Coverage","Total: with Health Insurance Coverage: Public Health Coverage","Total: with Health Insurance Coverage: Private Health Insurance","W","ST","STATE","ST_ABBR","STCNTY","COUNTY","LOCATION","M_TOTPOP","E_HU","M_HU","E_HH","M_HH","E_POV150","M_POV150","E_UNEMP","M_UNEMP","E_HBURD","M_HBURD","E_NOHSDP","M_NOHSDP","E_UNINSUR","M_UNINSUR","E_AGE65","M_AGE65","E_AGE17","M_AGE17","E_DISABL","M_DISABL","E_SNGPNT","M_SNGPNT","E_LIMENG","M_LIMENG","E_MINRTY","M_MINRTY","E_MUNIT","M_MUNIT","E_MOBILE","M_MOBILE","E_CROWD","M_CROWD","E_NOVEH","M_NOVEH","E_GROUPQ","M_GROUPQ","MP_POV150","MP_UNEMP","MP_HBURD","MP_NOHSDP","MP_UNINSUR","MP_AGE65","MP_AGE17","MP_DISABL","MP_SNGPNT","MP_LIMENG","MP_MINRTY","MP_MUNIT","MP_MOBILE","MP_CROWD","MP_NOVEH","MP_GROUPQ","E_NOINT","M_NOINT","E_AFAM","M_AFAM","E_HISP","M_HISP","E_ASIAN","M_ASIAN","E_AIAN","M_AIAN","E_NHPI","M_NHPI","E_TWOMORE","M_TWOMORE","E_OTHERRACE","M_OTHERRACE","MP_NOINT","MP_AFAM","MP_HISP","MP_ASIAN","MP_AIAN","MP_NHPI","MP_TWOMORE","MP_OTHERRACE","Count_Medical_Facilities","Count_Transportation_Stops"]

## Reduce number of columns

In [4]:
df_scaled=pd.read_csv('df_counties.csv')
#files.download('df_scaled.csv')

In [7]:
X= df_scaled[socioeconomic+spatial]
y = df_scaled['IDX_W2M']
x= sm.add_constant(X)
slr = sm.OLS(y,X).fit()
slr_summary=slr.summary()
print(slr.summary())

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)



Intel MKL ERROR: Parameter 4 was incorrect on entry to DLASCL.

Intel MKL ERROR: Parameter 5 was incorrect on entry to DLASCL.

Intel MKL ERROR: Parameter 4 was incorrect on entry to DLASCL.

Intel MKL ERROR: Parameter 5 was incorrect on entry to DLASCL.

Intel MKL ERROR: Parameter 4 was incorrect on entry to DLASCL.

Intel MKL ERROR: Parameter 4 was incorrect on entry to DLASCL.


LinAlgError: SVD did not converge

In [ ]:
vif = VIF(df_scaled[socioeconomic], 10).round(2)
vif

In [ ]:
reduced_features=list(vif['features'])
reduced_features

In [ ]:
vif.to_excel('vif.xlsx')

### Feature engineering

In [ ]:
df = df_scaled[targets].copy()

In [ ]:
features=[]
for s in spatial:
  for p in reduced_features:
    df[s + "*" + p]= df_scaled[s]*df_scaled[p]
    features.append(s + "*" + p)

In [ ]:
df.to_csv('df_engin.csv')

In [ ]:
len(features)

In [ ]:
X= df[features]
y = df['IDX_W2M']
x= sm.add_constant(X)
slr = sm.OLS(y,X).fit()
slr_summary=slr.summary()
print(slr.summary())

In [ ]:
pred=slr.predict(X)
mse = mean_squared_error(df['IDX_W2M'], pred)
rmse = mean_squared_error(df['IDX_W2M'], pred, squared=False)
mape = mean_absolute_percentage_error(df['IDX_W2M'], pred)
table = pd.DataFrame({'MSE':[mse], 'RMSE':[rmse], 'MAPE':[mape]})
table

In [ ]:
slr_as_html = slr_summary.tables[1].as_html()

stat=pd.read_html(slr_as_html, header=0, index_col=0)[0]

stat

In [ ]:
list(stat.columns)

In [ ]:
stat.loc[stat['P>|t|'] <= 0.01]

In [ ]:
stat

In [ ]:
significant_features=list(stat.index)

In [ ]:
len(stat)

In [ ]:
X= df[significant_features]
y = df['IDX_W2M']
x= sm.add_constant(X)
slr = sm.OLS(y,X).fit()
slr_summary=slr.summary()
print(slr.summary())

### Model without density variables

In [ ]:
spatial       = [#'Population Density (Per Sq. Mile)',#'Area (Land)',
    'Average Commute to Work (In Min)',# 'AREA_SQMI',
                 'Density_Medical_Facilities_Population',
                 'Density_Medical_Facilities_Area'
                 #'Density_Transportation_Stops_Pop',
                 #'Density_Transportation_Stops_Area'
                 #'Stops_per_Facility'
                ]

In [ ]:
features=[]
for s in spatial:
  for p in reduced_features:
    #df[s + "*" + p]= df_scaled[s]*df_scaled[p]
    features.append(s + "*" + p)

In [ ]:
X= df[features]
y = df['IDX_W2M']
x= sm.add_constant(X)
slr = sm.OLS(y,X).fit()
slr_summary=slr.summary()
print(slr.summary())

In [ ]:
pred=slr.predict(X)

In [ ]:
mse = mean_squared_error(df['IDX_W2M'], pred)
rmse = mean_squared_error(df['IDX_W2M'], pred, squared=False)
mape = mean_absolute_percentage_error(df['IDX_W2M'], pred)
table = pd.DataFrame({'MSE':[mse], 'RMSE':[rmse], 'MAPE':[mape]})
table